### 必要なライブラリ等のインポート

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

### 手順1：HTMLから欲しいテキストを抽出する

In [6]:
url = "https://magi.camp/blogs/167305889"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# ページ全体のテキストを行単位で取得
lines = soup.get_text(separator="\n").split("\n")

### 手順2：抽出したテキストから「発売日・希望小売価格・カード枚数」を切り分ける

In [7]:
data = []
i = 0
while i < len(lines):
    line = lines[i].strip()
    if line.startswith("発売日"):
        release = re.search(r"発売日[:：] ?(\d{4}年\d{1,2}月\d{1,2}日)", line)
        price = re.search(r"([0-9,]+円)", lines[i+1]) if i+1 < len(lines) else None
        count = re.search(r"カード[／/]([0-9]+)枚", lines[i+2]) if i+2 < len(lines) else None

        if release and price and count:
            data.append({
                "発売日": release.group(1),
                "希望小売価格": price.group(1),
                "カード枚数": count.group(1)
            })
            i += 3  # 次のセットへ
        else:
            i += 1  # マッチしなければ次行へ
    else:
        i += 1

### 手順3：切り分けたデータをデータフレームにまとめて、CSVで出力する

In [8]:
df = pd.DataFrame(data)
df.to_csv("pokemon_card_data.csv", index=False, encoding="utf-8-sig")
df.head()  

,発売日,希望小売価格,カード枚数
0,1996年10月20日,300円,10
1,1997年3月5日,300円,10
2,1997年6月21日,300円,10
3,1997年11月21日,300円,10
4,1998年10月24日,300円,10
